<a href="https://colab.research.google.com/github/proygit/DMT/blob/main/DMT_Assgnt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Change directories and load sample data

In [147]:
!ls /content


drive  sample_data


In [148]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [149]:
!ls /content/drive/MyDrive/dmt-data

DMT.csv  ODI-2023.xlsx


In [150]:
import os


# Change the working directory to the desired folder
os.chdir('/content/drive/MyDrive/dmt-data')

# List the contents of the current working directory
contents = os.listdir()

# Print the contents
print(contents)



['ODI-2023.xlsx', 'DMT.csv']


# EDA with excel

In [151]:
import pandas as pd

# Read in the Excel file
data_df = pd.read_excel('ODI-2023.xlsx')

# Create a copy of the DataFrame
df_cp = data_df.copy()

# Get the shape of the DataFrame
print(df_cp.shape)



(305, 17)


In [ ]:

df_cp.head(20)


# Explore Colums

In [ ]:
df_cp.columns


In [ ]:
df_cp.dtypes


In [ ]:
df_cp.describe()

In [156]:
# check null and isNaN values if any
print(df_cp.columns.isna().sum())
print(df_cp.columns.isnull().sum())

0
0


In [157]:
df_cp.to_csv('DMT.csv',index =False, sep=',', header=True)
# Print the first few rows of the DataFrame
df_csv = pd.read_csv('DMT.csv')
#print(df_csv.columns)
print(df_csv[['What is your gender?']].head(2))



  What is your gender?
0         gender fluid
1                 male


#check original rows of data comment the ones not important and convert object type to specific data types

In [158]:
print(df_cp.shape)
# original data set has 305 rows and 17 colums


(305, 17)


In [ ]:
print(df_cp.columns)

# Create a Table with Gender and if they used ChatGPT or NOT?

In [160]:
df_csv_ver1 =df_csv[[#'Tijdstempel', 
        #'What programme are you in?',
       #'Have you taken a course on machine learning?',
       #'Have you taken a course on information retrieval?',
       #'Have you taken a course on statistics?',
       #'Have you taken a course on databases?', 
        'What is your gender?',
       'I have used ChatGPT to help me with some of my study assignments ',
      # 'When is your birthday (date)?',
       #'How many students do you estimate there are in the room?',
       #'Did you stand up to come to your previous answer    ?',
       #'What is your stress level (0-100)?',
       #'How many hours per week do you do sports (in whole hours)? ',
       #'Give a random number', 
        #'Time you went to bed Yesterday',
       #'What makes a good day for you (1)?',
       #'What makes a good day for you (2)?'
       ]].iloc[:, :17]

In [161]:
df_csv_ver1.shape

(305, 2)

# Create a new table with data for students who took IR,Stats,ML before




In [ ]:
from sklearn.preprocessing import LabelEncoder
df_csv_ver2 =df_csv[[ 'What programme are you in?',
       'Have you taken a course on machine learning?',
       'Have you taken a course on information retrieval?',
       'Have you taken a course on statistics?',
       'Have you taken a course on databases?', 
        #'What is your gender?',
       #'I have used ChatGPT to help me with some of my study assignments ',
      # 'When is your birthday (date)?',
       #'How many students do you estimate there are in the room?',
       #'Did you stand up to come to your previous answer    ?',
       #'What is your stress level (0-100)?',
       #'How many hours per week do you do sports (in whole hours)? ',
       #'Give a random number', 
        #'Time you went to bed Yesterday',
       #'What makes a good day for you (1)?',
       #'What makes a good day for you (2)?'
       ]].iloc[:, :17]
df_csv_ver2.shape
#Remane the columns to a smaller name
df_csv_ver2 = df_csv_ver2.rename(columns={'What programme are you in?': 'Study Track','Have you taken a course on machine learning?':'taken ML before?'})
df_csv_ver2 = df_csv_ver2.rename(columns={'Have you taken a course on information retrieval?': 'taken IR before?','Have you taken a course on statistics?':'taken Statistics before?','Have you taken a course on databases?':'taken DB before?'})
df_csv_ver2

# Define the valid study tracks
study_tracks = ['Business Analytics','Bioinformatics','Bioinformatics & Systems Biology','Econometrics','Artificial Intelligence', 'AI', 'Masters - AI','M.S. Artificial Intelligence','Computational Science','BA','Econometrics and Data Science','CS']

# Convert all study tracks in the list to lowercase
study_tracks_lower = [s.lower() for s in study_tracks]

# Iterate over each row in the dataframe
for index, row in df_csv_ver2.iterrows():
    # Convert the study track to lowercase
    study_track_lower = str(row['Study Track']).lower()
    
    # If the study track is not in the list, mark it as NaN
    if study_track_lower not in study_tracks_lower:
        df_csv_ver2.loc[index, 'Study Track'] = np.nan

df_csv_ver2
#check how many we lose by having Nan
df_csv_ver2['Study Track'].isna().value_counts()
#after cleaningf study track we have 136 data of students in different tracks.
#So drop all other row in the csv for other tracks which are NaN
df_csv_ver2.dropna(inplace=True)
df_csv_ver2['Study Track'].isna().value_counts()
df_csv_ver2
# Replace "unknown" with 0 in 'taken IR before?' column
df_csv_ver2['taken IR before?'] = df_csv_ver2['taken IR before?'].replace('unknown', 0)
# Replace "unknown" with "sigma" in 'taken Statistics before?' column
df_csv_ver2['taken Statistics before?'] = df_csv_ver2['taken Statistics before?'].replace('unknown', 'sigma')
#Apply LabelEncoding values 1=yes,mu and ja AND 0 = no,sigma,nee
le = LabelEncoder()
# Encode the 'yes' and 'no' values in the 'taken IR before?' column as binary 1 and 0, respectively
df_csv_ver2['taken ML before?'] = le.fit_transform(df_csv_ver2['taken ML before?'])
df_csv_ver2['taken Statistics before?'] = le.fit_transform(df_csv_ver2['taken Statistics before?'])
df_csv_ver2['taken DB before?'] = le.fit_transform(df_csv_ver2['taken DB before?'])
#check after cleaning
df_csv_ver2



# Count of  Each Track and the Courses taken

In [ ]:
#Get a count of ML,IR,Statistics and DB for each study track
# Group the dataframe by study track
grouped = df_csv_ver2.groupby('Study Track')

# Get the counts for each column within each group
counts = grouped[['taken ML before?', 'taken IR before?', 'taken Statistics before?', 'taken DB before?']].sum()

# Rename the columns to reflect the count of 'yes' responses
counts = counts.rename(columns={
    'taken ML before?': 'ML Count',
    'taken IR before?': 'IR Count',
    'taken Statistics before?': 'Statistics Count',
    'taken DB before?': 'DB Count'
})

# Add a column for the total number of respondents in each group
counts['Total'] = grouped.size()

# Sort the dataframe by total number of respondents
counts = counts.sort_values('Total', ascending=False)

# Print the resulting dataframe
print(counts)


# Find the Percentage of each track who took either ML,IR,Statistics or DB

In [ ]:
import pandas as pd

# create dataframe from the data
data = {'Study Track': ['AI', 'BA', 'CS', 'Bioinformatics', 'Business Analytics', 'Artificial Intelligence',
                        'Econometrics', 'Computational Science', 'Econometrics and Data Science',
                        'Computational science', 'bioinformatics & systems biology', 'bioinformatics',
                        'Masters - AI', 'M.S. Artificial Intelligence', 'Artificial intelligence', 'Ai',
                        'Business analytics', 'Bioinformatics & Systems biology', 'Bioinformatics & Systems Biology',
                        'computational science'],
        'ML Count': [32, 19, 11, 11, 11, 7, 4, 5, 2, 2, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0],
        'Statistics Count': [5, 0, 5, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        'DB Count': [16, 4, 2, 7, 2, 6, 8, 8, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1],
        'Total': [43, 19, 16, 12, 11, 11, 11, 10, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

df = pd.DataFrame(data)

# calculate the percentage of each course for each study track
df['ML %'] = (df['ML Count'] / df['Total']) * 100
df['IR %'] = (df['Statistics Count'] / df['Total']) * 100
df['DB %'] = (df['DB Count'] / df['Total']) * 100
df['Statistics %'] = (df['IR %'] + df['DB %'])  # calculate the percentage of Statistics

# print the resulting dataframe
print(df[['Study Track', 'ML %', 'IR %', 'DB %', 'Statistics %']])


# Plot a graph to show if students from different tracks took which courses before - IR,ML,Statistics or DB  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
labels = ['AI', 'BA', 'CS', 'Bioinformatics', 'Business Analytics', 'Artificial Intelligence', 'Econometrics', 'Computational Science', 'Econometrics and Data Science']
ml = [32, 19, 11, 11, 11, 7, 4, 5, 2]
ir = [5, 0, 5, 2, 0, 1, 1, 1, 0]
stats = [16, 4, 2, 7, 2, 6, 8, 8, 1]
db = [43, 19, 16, 12, 11, 11, 11, 10, 3]

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(labels))
ax.bar(x, ml, label='Machine Learning', color='#1f77b4')
ax.bar(x, ir, bottom=ml, label='Information Retrieval', color='#ff7f0e')
ax.bar(x, stats, bottom=np.add(ml, ir), label='Statistics', color='#2ca02c')
ax.bar(x, db, bottom=np.add(np.add(ml, ir), stats), label='Databases', color='#d62728')

# Customize plot
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=90)
ax.set_ylabel('Count')
ax.set_title('Course history by study track')
ax.legend()

plt.show()


#Drop Rows which are not Male or Female

In [ ]:
#Original count of rows and columns are - 305 row and 2 columns
df_csv_ver1.shape
#check values of Gender if there are Male or Female or anything else
df_csv_ver1['What is your gender?'].value_counts()
# if the values are not male or female convert them to NaN
mask = ~ df_csv_ver1['What is your gender?'].isin(['male', 'female'])
df_csv_ver1['What is your gender?'].value_counts()
df_csv_ver1


In [ ]:
import numpy as np
df_csv_ver1.loc[mask, 'What is your gender?'] = np.nan
df_csv_ver1['What is your gender?'].value_counts()
df_csv_ver1
# drop the NaN row values 
df_csv_ver1.dropna(inplace=True)
df_csv_ver1


# Plot a graph with the above value

In [168]:
import matplotlib.pyplot as plt
df_csv_ver1['What is your gender?'].value_counts()


male      154
female    134
Name: What is your gender?, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
df_csv_ver1.shape
df_csv_ver1
#Convert categorial to numeric values
le = LabelEncoder()
df_csv_ver1['What is your gender?'] = le.fit_transform(df_csv_ver1['What is your gender?'])
#df_csv_ver1['I have used ChatGPT to help me with some of my study assignments '] = le.fit_transform(df_csv_ver1['I have used ChatGPT to help me with some of my study assignments '])
# Rename the column since its too long 
df_csv_ver1 = df_csv_ver1.rename(columns={'I have used ChatGPT to help me with some of my study assignments ': 'Used ChatGPT?'})

df_csv_ver1



In [ ]:

# group the data by gender and the 3 types of answers
df_grouped = df_csv_ver1.groupby(['What is your gender?', 'Used ChatGPT?']).size().reset_index(name='Count')

# create a pie chart for male and female answers
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

df_male = df_grouped[df_grouped['What is your gender?'] == 0]
df_female = df_grouped[df_grouped['What is your gender?'] == 1]

ax[0].pie(df_male['Count'], labels=df_male['Used ChatGPT?'], colors=colors, autopct='%1.1f%%')
ax[0].set_title('Male Responses')

ax[1].pie(df_female['Count'], labels=df_female['Used ChatGPT?'], colors=colors, autopct='%1.1f%%')
ax[1].set_title('Female Responses')

fig.suptitle('Percentages of Male and Female Responses by Answer Type')

plt.show()


# Grouped Bar Plot for Male and Female answers if they used ChatGPT

In [ ]:
df_male = df_csv_ver1[df_csv_ver1['What is your gender?'] == 1]
df_female = df_csv_ver1[df_csv_ver1['What is your gender?'] == 0]

male_yes = df_male[df_male['Used ChatGPT?'] == 'yes']['Used ChatGPT?'].count()
male_no = df_male[df_male['Used ChatGPT?'] == 'no']['Used ChatGPT?'].count()
male_not_say = df_male[df_male['Used ChatGPT?'] == 'not willing to say']['Used ChatGPT?'].count()

female_yes = df_female[df_female['Used ChatGPT?'] == 'yes']['Used ChatGPT?'].count()
female_no = df_female[df_female['Used ChatGPT?'] == 'no']['Used ChatGPT?'].count()
female_not_say = df_female[df_female['Used ChatGPT?'] == 'not willing to say']['Used ChatGPT?'].count()

# Set up the data for the bar plot
answer_types = ['Yes', 'No', 'Not willing to say']
male_values = [male_yes, male_no, male_not_say]
female_values = [female_yes, female_no, female_not_say]
x = np.arange(len(answer_types))
bar_width = 0.35

# Create the grouped bar plot
fig, ax = plt.subplots()
rects1 = ax.bar(x - bar_width/2, male_values, bar_width, label='Male')
rects2 = ax.bar(x + bar_width/2, female_values, bar_width, label='Female')

# Add some text for labels, title and axes ticks
ax.set_xlabel('Answer Type')
ax.set_ylabel('Count by Gender Type')
ax.set_title('Used ChatGPT? by Gender')
ax.set_xticks(x)
ax.set_xticklabels(answer_types)
ax.legend()

# Display the plot
plt.show()